In [4]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [13]:
DEG90INRAD = np.pi * 0.5 # 1.5707963267948966;
DEG180INRAD = np.pi       # 3.1415926535897931;
DEG270INRAD = np.pi * 1.5 # 4.7123889803846897;
DEG360INRAD = np.pi * 2   # 6.2831853071795862;

In [7]:
class Configuration:
    # precision of angle values -> 0.0175 ~ 1 degree; 0.0873 ~ 5 degree
    steering_angle_tolerance = 0.0175 # (0., 0.0873)
    # angle, that has to be considered in both directions, when driving forward; 1.5707 ~ 90 degree
    half_angle_front_init = 0.63 # (0., 1.5707)
    # angle, that has to be considered in both directions, when driving backward; 1.5707 ~ 90 degree
    half_angle_back_init = 0.33 # (0., 1.5707)
    # distance from lidar (center) to trailing edge
    reverse_minimum_distance = 0.29 # (0.0, 1.0)
    # distance from lidar (center) to leading edge
    forward_minimum_distance = 0.13 # (0.0, 1.0)
    # points of what max. distance should be considered at all
    maximum_lookahead_distance = 3.0 # (1.0, 6.0)
    # car stops at that distance, independent of any other circumstances
    stop_distance = 0.05 # (0.0, 2.0)
    # distance to an obstacle, from which on the wanted speed gets damped linear
    max_startup_damp_range = 3.0 # (0.0, 3.0)
    # avg. deceleration of that specific car
    negative_acceleration = 0.5 # (0.3, 1.0)
    # value ...
    target_quotient = 1.0 # (1.0, 1.5)
    # distance from front to rear axle
    wheelbase = 0.26 # (0., 0.5)
    # distance of both middles of the tire widths of rear tires
    track = 0.165 # (0.1, 0.3)
    # distance from lidar (center) to rear axle
    lidar_rear_axle_distance = 0.2 # (0.05, 0.4)
    # total length of the car, including bumper etc
    car_length = 0.43 # (0.3, 0.5)
    # total width of the car
    car_width = 0.19 # (0.15, 0.25)

In [8]:
config = Configuration()

In [15]:
# Library
def hypotenuse(a, b):
    return np.sqrt(np.power(a, 2) + np.power(b, 2))

def getDistanceToCar(distanceToLidar, deg_step):
    if deg_step < 90 or deg_step > 270:
        return(distanceToLidar - config.forward_minimum_distance)
    else:
        return(distanceToLidar - config.reverse_minimum_distance)

def getTurningRadius(steeringAngle):
    return(config.wheelbase / np.tan(steeringAngle))

def getX(radius, alpha, d):
    t = np.cos(alpha) * d
    h = np.sin(alpha) * d
    s = radius - t
    tan_beta = h / s
    beta = np.arctan(tan_beta)
    x = s / np.cos(beta)
    return x

def isOnPath(radius, x):
    return(x < (radius + config.car_width/2) and x > (radius - config.car_width/2))

def calculateSafeSpeed(distance, deceleration, targetQuotient):
    return np.sqrt(distance * deceleration / targetQuotient)

def safeDistanceQuotient(distance, deceleration, currentSpeed):
    if currentSpeed == 0.0:
        return float('inf')
    return distance * deceleration / np.power(currentSpeed, 2)

# works on actual lidar position & values
def isOnStraightPath(posAngle, distance, backward):
    distanceFromMiddle = np.sin(posAngle) * distance
    safetyWidth = 0.02
    if backward:
        safetyWidth = 0.04
    return(distanceFromMiddle < ((config.car_width + safetyWidth) / 2))

def projectOnRearAxle(angle, distance, offset=config.lidar_rear_axle_distance):
    newAngle = np.arctan((np.sin(angle) * distance) / (np.cos(angle) * distance + offset))
    newDistance = np.sqrt(np.power(np.sin(angle) * distance, 2) + np.power(np.cos(angle) * distance + offset, 2))
    return (newAngle, newDistance)

def getProjectedAngle(angle, distance, offset=config.lidar_rear_axle_distance):
    return(np.arctan((np.sin(angle) * distance) / (np.cos(angle) * distance + offset)))

def getProjectedDistance(angle, distance, offset=config.lidar_rear_axle_distance):
    return(np.sqrt(np.power(np.sin(angle) * distance, 2) + np.power(np.cos(angle) * distance + offset, 2)))


In [10]:
getTurningRadius(np.deg2rad(30))

0.45033320996790815

In [6]:
def projectOnRearAxle(angle, distance, offset=config.lidar_rear_axle_distance):
    deg90 = 1.5707963267948966
    deg180 = 3.1415926535897931
    deg270 = 4.7123889803846897
    deg360 = 6.2831853071795862
    
    if angle >= 0.0 and angle < deg90: #  0 - 90 deg
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        newAngle = np.arctan(a / (b + offset))
        newDistance = np.sqrt(np.power(a, 2) + np.power(b + offset, 2))
        return (newAngle, newDistance)
    
    if angle >= deg90 and angle < deg180: #  90 - 180 deg
        angle = deg180 - angle
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        if b > offset:
            newAngle = np.arctan(a / (b - offset))
            newDistance = np.sqrt(np.power(a, 2) + np.power(b - offset, 2))
            return (deg180 - newAngle, newDistance)
        elif b < offset:
            newAngle = np.arctan(a / (offset - b))
            newDistance = np.sqrt(np.power(a, 2) + np.power(offset - b, 2))
            return (newAngle, newDistance)
        else:
            return (deg90, a)
    
    if angle >= deg180 and angle < deg270: # 180 - 270 deg
        angle = angle - deg180
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        if b > offset:
            newAngle = np.arctan(a / (b - offset))
            newDistance = np.sqrt(np.power(a, 2) + np.power(b - offset, 2))
            return (deg180 + newAngle, newDistance)
        elif b < offset:
            newAngle = np.arctan(a / (offset - b))
            newDistance = np.sqrt(np.power(a, 2) + np.power(offset - b, 2))
            return (deg360 - newAngle, newDistance)
        else:
            return (deg270, a)
    
    if angle >= deg270 and angle < deg360: # 270 - 360 deg
        angle = deg360 - angle
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        newAngle = np.arctan(a / (b + offset))
        newDistance = np.sqrt(np.power(a, 2) + np.power(b + offset, 2))
        return (deg360 - newAngle, newDistance)
    
    


In [7]:
def projectOnRearAxleAngle(angle, distance, offset=config.lidar_rear_axle_distance):
    deg90 = 1.5707963267948966
    deg180 = 3.1415926535897931
    deg270 = 4.7123889803846897
    deg360 = 6.2831853071795862
    
    if angle >= 0.0 and angle < deg90: #  0 - 90 deg
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        return np.arctan(a / (b + offset))
    
    if angle >= deg90 and angle < deg180: #  90 - 180 deg
        angle = deg180 - angle
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        if b > offset:
            return (deg180 - (np.arctan(a / (b - offset))))
        else:
            return np.arctan(a / (offset - b))
            
    
    if angle >= deg180 and angle < deg270: # 180 - 270 deg
        angle = angle - deg180
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        if b > offset:
            return (deg180 + np.arctan(a / (b - offset)))
        else:
            return (deg360 - np.arctan(a / (offset - b)))
    
    if angle >= deg270 and angle < deg360: # 270 - 360 deg
        angle = deg360 - angle
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        return (deg360 - np.arctan(a / (b + offset)))
    
    
def projectOnRearAxleDist(angle, distance, offset=config.lidar_rear_axle_distance):
    deg90 = 1.5707963267948966
    deg180 = 3.1415926535897931
    deg270 = 4.7123889803846897
    deg360 = 6.2831853071795862
    
    if angle >= 0.0 and angle < deg90: #  0 - 90 deg
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        return np.sqrt(np.power(a, 2) + np.power(b + offset, 2))
    
    if angle >= deg90 and angle < deg180: #  90 - 180 deg
        angle = deg180 - angle
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        if b > offset:
            return np.sqrt(np.power(a, 2) + np.power(b - offset, 2))
        else:
            return np.sqrt(np.power(a, 2) + np.power(offset - b, 2))            
    
    if angle >= deg180 and angle < deg270: # 180 - 270 deg
        angle = angle - deg180
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        if b > offset:
            return np.sqrt(np.power(a, 2) + np.power(b - offset, 2))
        else:
            return np.sqrt(np.power(a, 2) + np.power(offset - b, 2))
    
    if angle >= deg270 and angle < deg360: # 270 - 360 deg
        angle = deg360 - angle
        a = np.sin(angle) * distance
        b = np.cos(angle) * distance
        return np.sqrt(np.power(a, 2) + np.power(b + offset, 2))
    
    


In [27]:
def getExactDistanceToCar(rad, dist):
        # works with actual lidar, not with the projected!
        offsetFront = config.forward_minimum_distance
        offsetRear = config.reverse_minimum_distance
        angleFront = config.half_angle_front_init
        angleRear = config.half_angle_back_init
        
        print("rad: %f\noffsetFront: %f\nangleFront: %f" % (rad, offsetFront, angleFront))

        # front: approx. same distance -> because of radial bumper
        if (angleFront >= rad or rad >= (DEG360INRAD - angleFront)):
            print("1\n")
            return dist - offsetFront
        
        elif (angleFront < rad and rad <= DEG90INRAD):
            print("2\n")
            alpha = DEG90INRAD - rad
            b = config.car_width / 2
            a = b * np.tan(alpha)
            c = hypotenuse(a, b)
            return dist - c
        
        elif (DEG90INRAD < rad and rad < (DEG180INRAD - angleRear)):
            print("3\n")
            alpha = rad - DEG90INRAD
            b = config.car_width / 2
            a = b * np.tan(alpha)
            c = hypotenuse(a, b)
            return dist - c
        
        elif ((DEG180INRAD - angleRear) <= rad and rad < DEG180INRAD):
            print("4\n")
            alpha = DEG180INRAD - rad
            b = offsetRear
            a = b * np.tan(alpha)
            c = hypotenuse(a, b)
            return dist - c
        
        elif (DEG180INRAD <= rad and rad <= (DEG180INRAD + angleRear)):
            print("5\n")
            alpha = rad - DEG180INRAD
            b = offsetRear
            a = b * np.tan(alpha)
            c = hypotenuse(a, b)
            return dist - c
        
        elif ((DEG180INRAD + angleRear) < rad and rad <= DEG270INRAD):
            print("6\n")
            alpha = DEG270INRAD - rad
            b = config.car_width / 2
            a = b * np.tan(alpha)
            c = hypotenuse(a, b)
            return dist - c
        
        elif (DEG270INRAD < rad and rad < (DEG360INRAD - angleFront)):
            print("7\n")
            alpha = rad - DEG270INRAD
            b = config.car_width / 2
            a = b * np.tan(alpha)
            c = hypotenuse(a, b)
            return dist - c
        
        else:
            return np.Inf
        

In [32]:
getExactDistanceToCar(config.half_angle_front_init, 0.20)#getExactDistanceToCar(np.deg2rad(90.000), 0.20)

rad: 0.630000
offsetFront: 0.130000
angleFront: 0.630000
1



0.07

In [9]:
for i in np.arange(0, 360, 1):
    p = projectOnRearAxle(np.deg2rad(i), 0.5)
    print "Winkel: ", i
    print(np.rad2deg(p[0]))
    print(p[1])
    print(15*"-")

Winkel:  0
0.0
0.7
---------------
Winkel:  1
0.714290154871
0.699978241827
---------------
Winkel:  2
1.4286069556
0.699912969878
---------------
Winkel:  3
2.14297705983
0.699804191864
---------------
Winkel:  4
2.8574271487
0.699651920638
---------------
Winkel:  5
3.57198393871
0.699456174194
---------------
Winkel:  6
4.28667419349
0.699216975676
---------------
Winkel:  7
5.00152473567
0.698934353375
---------------
Winkel:  8
5.71656245878
0.698608340738
---------------
Winkel:  9
6.43181433921
0.698238976368
---------------
Winkel:  10
7.14730744825
0.697826304034
---------------
Winkel:  11
7.86306896415
0.697370372678
---------------
Winkel:  12
8.57912618434
0.696871236418
---------------
Winkel:  13
9.29550653768
0.696328954559
---------------
Winkel:  14
10.0122375969
0.695743591602
---------------
Winkel:  15
10.7293470909
0.695115217254
---------------
Winkel:  16
11.4468629178
0.694443906437
---------------
Winkel:  17
12.1648131571
0.693729739302
---------------
Winkel

In [10]:
np.deg2rad(90)

1.5707963267948966

In [11]:
np.deg2rad(180)

3.1415926535897931

In [12]:
np.deg2rad(89.999999)

1.5707963093416042

In [13]:
np.deg2rad(90)

1.5707963267948966

In [14]:
#steerAngMin = -0.785398
#steerAngMax = 0.785398

In [15]:
steerDeg = 30

In [16]:
steerAngMin = -np.deg2rad(steerDeg)
steerAngMax = np.deg2rad(steerDeg)

In [17]:
steeringAngles = np.arange(steerAngMin, steerAngMax, 0.0175)

In [18]:
#print("0 Deg:    " + str(getTurningRadius(0)))
print("1 Deg:    " + str(getTurningRadius(0.0175)))
for stan in steeringAngles:
    print(str(np.rad2deg(np.abs(stan))) + ":    " + str(getTurningRadius(np.abs(stan))))

1 Deg:    14.8556261595
30.0:    0.450333209968
28.9973238585:    0.46910408821
27.994647717:    0.489099097716
26.9919715756:    0.510455541488
25.9892954341:    0.533331871688
24.9866192926:    0.557911933952
23.9839431511:    0.584410276279
22.9812670096:    0.613078847645
21.9785908682:    0.64421553013
20.9759147267:    0.678175118087
19.9732385852:    0.715383604242
18.9705624437:    0.756356996244
17.9678863023:    0.801726433366
16.9652101608:    0.852272209826
15.9625340193:    0.908970620971
14.9598578778:    0.973059648431
13.9571817363:    1.04613295794
12.9545055949:    1.13027754954
11.9518294534:    1.2282806952
10.9491533119:    1.34395057898
9.94647717042:    1.48263087423
8.94380102894:    1.65206145136
7.94112488746:    1.86389114288
6.93844874598:    2.13650216735
5.93577260451:    2.50069717905
4.93309646303:    3.01232184247
3.93042032155:    3.78420801865
2.92774418007:    5.08375520813
1.92506803859:    7.73546511558
0.922391897111:    16.1489012984
0.0802842443

In [19]:
np.tan(0.85398)

1.1475116368338036

In [20]:
angle = np.deg2rad(93)
d = 3

In [21]:
alpha, dist = projectOnRearAxle(angle, d)
print(np.rad2deg(alpha))
print(dist)

89.1778403904
2.99619706503


In [22]:
np.rad2deg(0.0785)

4.4977186917769627

In [23]:
# Grenzwinkel vorne und hinten projected Lidar

In [24]:
np.rad2deg(0.63)

36.096341093241861

In [25]:
np.rad2deg(0.33)

18.907607239317166

In [26]:
np.sqrt(np.power(13, 2) + np.power(9.5, 2))

16.101242188104617

In [27]:
# front-edge:

In [28]:
np.sqrt(np.power(33, 2) + np.power(9.5, 2))

34.340209667385551

In [29]:
alpha_front = np.arcsin(9.5 / np.sqrt(np.power(33, 2) + np.power(9.5, 2)))
print(alpha_front)
print(np.rad2deg(alpha_front))

0.280299656338
16.0599873072


In [30]:
# rear-edge:

In [31]:
np.sqrt(np.power(10, 2) + np.power(9.5, 2))

13.793114224133722

In [32]:
alpha_back = np.arcsin(9.5 / np.sqrt(np.power(10, 2) + np.power(9.5, 2)))
print(alpha_back)
print(np.rad2deg(alpha_back))

0.759762754876
43.5311992856


In [33]:
np.rad2deg(0.76)

43.544792429942568

In [34]:
# ----------------

In [35]:
90 - np.rad2deg(0.28)

73.957181736336949

In [36]:
0.26 / np.tan(np.pi / 6)

0.45033320996790815

In [45]:
np.sqrt((config.car_width/2)**2 + config.lidar_rear_axle_distance**2)

0.22141589825484531

In [46]:
np.arcsin(config.lidar_rear_axle_distance / np.sqrt((config.car_width/2)**2 + config.lidar_rear_axle_distance**2))

1.1273479903751584

In [53]:
np.rad2deg(2*np.pi - 0.63)

323.90365890675815

In [51]:
0.01745 * 360 - np.pi*2

-0.0011853071795862036

In [52]:
np.rad2deg(0.0011853071795862036)

0.067913098816844597